# Model Class

This notebook contains the AI model to finetune and generate the essay.

## Start Notebook
Run all of the command below to start the notebook training session of the model.

Uncomment and run the code below to kill the runtime in Google Colaboratory

In [ ]:
# !kill -9 -1


Uncomment and run the code below to install necessary file in Colaboratory

In [ ]:
# %tensorflow_version 1.x

# !pip install gpt_2_simple
# !pip install wikipedia

# !nvidia-smi


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')


In [ ]:
# !mkdir -p drive/My\ Drive/Project\ Writer/datasets
# !mkdir -p drive/My\ Drive/Project\ Writer/checkpoint
# !mkdir -p drive/My\ Drive/Project\ Writer/samples


Uncomment and run the code below to download the model in Colaboratory

In [ ]:
# import gpt_2_simple as gpt2
# gpt2.download_gpt2(model_name='355M')


Import necessary libraries

In [3]:
# Import libraries
import gpt_2_simple as gpt2
import tensorflow as tf


## Model
This function is used to finetune the model based on the current latest dataset. 

In [12]:
def finetune_model(dataset, any_checkpoint=False, reset_session=False):
    '''

    Function to finetune the model and save the trained model every checkpoint on the checkpoint folder.

    @dataset: Path to the training data (TXT) with minimum 1024 tokens
    @any_checkpoint: Boolean if there is any previous checkpoint
    @reset_session: Boolean if reseting the session graph is needed

    return: None

    '''

    # Parameters
    STEPS = 10000
    MODEL_NAME = '355M'
    LEARNING_RATE = 0.0001
    RUN_NAME = 'trained_model'

    MODEL_DIR = 'models'
    CHECKPOINT_DIR = 'checkpoint'

    # Clear session graph
    if reset_session:
        tf.reset_default_graph()

    # Initialize training session
    sess = gpt2.start_tf_sess()

    # Load the previous checkpoint
    if any_checkpoint:
        gpt2.load_gpt2(
            sess,
            run_name=RUN_NAME,
            checkpoint_dir=CHECKPOINT_DIR,
            model_name=None,
            model_dir=MODEL_DIR,
            multi_gpu=False
        )

    # Finetune the model
    gpt2.finetune(
        sess,
        dataset=dataset,  # Dataset file
        steps=STEPS,
        model_name=MODEL_NAME,  # Model name: 124M, 355M, etc.
        model_dir=MODEL_DIR,
        combine=50000,
        batch_size=1,
        learning_rate=LEARNING_RATE,  # Learning rate
        accumulate_gradients=5,
        restore_from='latest',  # Start training the model from the latest model
        run_name=RUN_NAME,  # Name of the trained model
        checkpoint_dir=CHECKPOINT_DIR,  # Directory to save the model
        sample_every=1000,
        sample_length=300,  # Number of token generated
        sample_num=1,
        multi_gpu=False,
        save_every=1000,
        print_every=10,
        max_checkpoints=1,
        use_memory_saving_gradients=False,
        only_train_transformer_layers=False,
        optimizer='adam',
        overwrite=True  # Overwrite the current model when training
    )


Uncomment and run the code below to test the finetuning model function

In [ ]:
# test_training_data = ''
# finetune_model(test_training_data)


## Generating Text
This functions is used to generate the text based on some input from the users.

In [ ]:
def generate_text(outline_to_length):
    '''

    Function to generate the text.

    @outline_to_length: A 2D array containing the list of outline and the length desired
        [[outline, length],
        [outline, length],
        [outline, length]]

    return: List of generated text

    '''

    # Parameters
    MODEL_NAME = '355M'
    RUN_NAME = 'trained_model'

    MODEL_DIR = 'models'
    CHECKPOINT_DIR = 'checkpoint'
    SAMPLE_DIR = 'samples'

    # Clear session graph
    tf.reset_default_graph()

    # Initialize TensorFlow session
    sess = gpt2.start_tf_sess()

    # Create an empty list to store lists
    essay = []

    # Loop over the list
    for record in outline_to_length:
        prefix = record[0]  # The first sentence of the paragraph
        length = record[1]  # The length of the paragraph (max: 1023)

        text = gpt2.generate(
            sess,
            run_name=RUN_NAME,
            checkpoint_dir=CHECKPOINT_DIR,
            model_name=None,
            model_dir=MODEL_DIR,
            sample_dir=SAMPLE_DIR,
            return_as_list=True,  # Return as list of string
            truncate=None,
            destination_path=None,
            sample_delim='\n\n' + '=' * 20 + '\n\n',
            prefix=prefix,
            seed=None,
            nsamples=1,  # Number of sample to be generated
            batch_size=1,
            length=length,
            temperature=0.7,
            top_k=0,
            top_p=0.0,
            include_prefix=True
        )

        text = ''.join(text) + '\n\n'
        essay += text

    return essay


Uncomment and run the code below to test the function

In [ ]:
# test_outline_to_length = [[]]
# print(generate_text(test_outline_to_length))
